In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import mean_squared_error

In [2]:
# 1. Environment Setup
np.random.seed(42)
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Preprocessing
# Generate random time-series data (1000 samples, 10 timesteps, 1 feature)
X = np.random.rand(1000, 10, 1).astype(np.float32)
y = np.random.rand(1000, 1).astype(np.float32)

X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)

dataset = TensorDataset(X_tensor, y_tensor)

# 3. Train-Test Split
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [3]:
# 4. Train Base Model (Long Short-Term Memory)
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use the last time step's output
        return out

model = LSTMModel(input_size=1, hidden_size=16, output_size=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training function
def train(model, loader, criterion, optimizer):
    model.train()
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

# Validation function
def evaluate(model, loader, criterion):
    model.eval()
    losses = []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            losses.append(loss.item())
    return np.mean(losses)

# Train base model
for epoch in range(5):
    train(model, train_loader, criterion, optimizer)
    val_loss = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")

Epoch 1, Validation Loss: 0.0964
Epoch 2, Validation Loss: 0.0879
Epoch 3, Validation Loss: 0.0876
Epoch 4, Validation Loss: 0.0885
Epoch 5, Validation Loss: 0.0872


In [4]:
# 5. Planning (Simulated Experience Using LSTM Predictions)
model.eval()
pseudo_labels = []
with torch.no_grad():
    for X_batch, _ in val_loader:
        X_batch = X_batch.to(device)
        preds = model(X_batch)
        pseudo_labels.append(preds.cpu())

pseudo_labels = torch.cat(pseudo_labels, dim=0)
X_augmented = torch.cat([train_dataset[:][0], val_dataset[:][0]], dim=0)
y_augmented = torch.cat([train_dataset[:][1], pseudo_labels], dim=0)

augmented_dataset = TensorDataset(X_augmented, y_augmented)
augmented_loader = DataLoader(augmented_dataset, batch_size=32, shuffle=True)

# 6. Fine-Tune Model
optimizer = optim.Adam(model.parameters(), lr=0.005)
for epoch in range(3):
    train(model, augmented_loader, criterion, optimizer)
    val_loss = evaluate(model, val_loader, criterion)
    print(f"Fine-Tune Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")

# 7. Evaluate
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch)
        y_true.append(y_batch.cpu().numpy())
        y_pred.append(preds.cpu().numpy())

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)
mse = mean_squared_error(y_true, y_pred)
print(f"Test MSE: {mse:.4f}")

# 8. Deploy Policy
model_path = "lstm_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved as '{model_path}'")

Fine-Tune Epoch 1, Validation Loss: 0.0873
Fine-Tune Epoch 2, Validation Loss: 0.0880
Fine-Tune Epoch 3, Validation Loss: 0.0882
Test MSE: 0.0824
Model saved as 'lstm_model.pth'
